# École Polytechnique de Montréal
# Département Génie Informatique et Génie Logiciel

# INF8460 – Traitement automatique de la langue naturelle - TP3

# Objectifs d’apprentissage
 • Utiliser des plongements lexicaux pré-entrainés pour de la classification
 
 • Entrainer des plongements lexicaux de type word2vec
 
 • Implanter des modèles de classification neuronaux

## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Cedric Sadeu (1869737): 1/3

Mamoudou Sacko (1924187): 1/3

Oumayma Messoussi (2016797): 1/3

# Librairies externes

In [1]:
import io
import os
import nltk
import time
import gensim
import sklearn
import zipfile
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from typing import Dict
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from collections import Counter, defaultdict
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score #, confusion_matrix, plot_confusion_matrix

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

# Téléchargement et lecture des données

In [3]:
DATA_PATH = os.path.join(os.getcwd(), "aclImdb")

## Téléchargement

In [4]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz
!rm aclImdb_v1.tar.gz
!echo Done!

--2020-10-21 17:40:32--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  3.12MB/s    in 37s     

2020-10-21 17:41:09 (2.16 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

Done!


In [5]:
def download_wikipedia_embeddings() -> None:
    if not os.path.exists(os.path.join(os.getcwd(), "model.txt")):
        res = requests.get("http://vectors.nlpl.eu/repository/11/3.zip")
        with zipfile.ZipFile(io.BytesIO(res.content)) as z:
            z.extractall("./")
        os.remove(os.path.join(os.getcwd(), "3.zip"))
        os.remove(os.path.join(os.getcwd(), "meta.json"))
        os.remove(os.path.join(os.getcwd(), "model.bin"))
        os.remove(os.path.join(os.getcwd(), "README"))

## Lecture

In [6]:
def read_data(path):
    traintest = ['train', 'test']
    classes = ['pos', 'neg']
    corpus = {cls: [] for cls in classes}

    # Each data is a list of strings(reviews)
    reviews = []
    labels = []
    for cls in classes:
        dir_path = os.path.join(path, cls)
        
        for filename in os.listdir(dir_path):
            file = os.path.join(dir_path, filename)
            with open(file, encoding = 'utf-8') as f:
                corpus[cls].append(f.read().replace("\n", " "))
        
    return corpus

In [7]:
train_data = read_data(os.path.join(DATA_PATH, 'train'))
test_data = read_data(os.path.join(DATA_PATH, 'test'))

In [8]:
train_data['pos'][0]

"I did not set very high expectations for this movie, which left me pleasantly surprised. The story is a little strange sometimes but overall I think it has an acceptable credibility. The action scenes are rather nice and the accompanying music is used to induce a a bit of patriotic feelings common to US movies. This may not be the best movie ever but it's uncommon for Sweden and I hope to see more similar ones in the future."

In [9]:
def create_wikipedia_embeddings(word_indices: Dict[str, int], vocab_len: int) -> np.ndarray:
    with open("./model.txt", "r", encoding="UTF-8") as f:
        shape_string = f.readline()
        lines = f.readlines() 
        
    embedding = np.zeros((vocab_len, 300), dtype=float)
    for line in lines:
        splitted_line = line.split(" ")
        word = splitted_line[0].split("_")[0]
        if word in word_indices and word_indices[word] < vocab_len:
            embedding_line = splitted_line[1:]
            embedding[word_indices[word]] = list(map(float, embedding_line))
        
    return embedding

## Prétraitement

In [10]:
class Preprocess(object):
    def __init__(self, lemmatize=True):
        self.stopwords = set(nltk.corpus.stopwords.words("english"))
        self.lemmatize = lemmatize

    def preprocess_pipeline(self, data):
        clean_tokenized_data = self._clean_doc(data)
        if self.lemmatize:
            clean_tokenized_data = self._lemmatize(clean_tokenized_data)

        return clean_tokenized_data

    def _clean_doc(self, data):
        tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
        return [
            [
                token.lower()
                for token in tokenizer.tokenize(review)
                if token.lower() not in self.stopwords
                and len(token) > 1
                and token.isalpha()
                and token != "br]"
            ]
            for review in data
        ]

    def _lemmatize(self, data):
        lemmatizer = nltk.stem.WordNetLemmatizer()
        return [[lemmatizer.lemmatize(word) for word in review] for review in data]

    def convert_to_reviews(self, tokenized_reviews):
        reviews = []
        for tokens in tokenized_reviews:
            reviews.append(" ".join(tokens))

        return reviews

In [11]:
pre = Preprocess()

train_pos = pre.preprocess_pipeline(train_data["pos"])
train_neg = pre.preprocess_pipeline(train_data["neg"])
test_pos = pre.preprocess_pipeline(test_data["pos"])
test_neg = pre.preprocess_pipeline(test_data["neg"])

y_train = [1] * len(train_pos) + [0] * len(train_neg)
y_test = [1] * len(test_pos) + [0] * len(test_neg)
X_train = [" ".join(sentence) for sentence in train_pos + train_neg]
X_test = [" ".join(sentence) for sentence in test_pos + test_neg]

print("{} training sentences: {} pos and {} neg".format(len(X_train), len(train_pos), len(train_neg)))
print("{} test sentences: {} pos and {} neg".format(len(X_test), len(test_pos), len(test_neg)))

25000 training sentences: 12500 pos and 12500 neg
25000 test sentences: 12500 pos and 12500 neg


# 1. Entrainement de plongements lexicaux

Vous devez réaliser les étapes suivantes:

## a) Utiliser Gensim pour entrainer un modèle word2vec sur le corpus. 

In [12]:
X_train_tokenized = [sentence for sentence in train_pos + train_neg]

model = Word2Vec(min_count=1, window=5, size=256, alpha=1e-2, min_alpha=1e-4, 
                 workers=(os.cpu_count()*2 - 1), sample=0.01, negative=5)

model.build_vocab(X_train_tokenized)

start = time.time()
model.train(X_train_tokenized, total_examples=model.corpus_count, epochs=10)
end = time.time() - start

## b) Décrire les paramètres du ou des modèles entraînés, leur taille sur disque, le nombre de mots encodés, le temps d'entraînement, etc.

In [13]:
print("- Temps d'entrainement (en secondes): %f\n" % end)
print("- Taille du modele sur disque (en octets): ", model.estimate_memory())

word_vectors = model.wv
print("\n- Nombre de mots encodés (= taille du vocab): %d\n" % len(word_vectors.vectors))

- Temps d'entrainement (en secondes): 71.511453

- Taille du modele sur disque (en octets):  {'vocab': 32849500, 'vectors': 67275776, 'syn1neg': 67275776, 'total': 167401052}

- Nombre de mots encodés (= taille du vocab): 65699



**Les parametres du modele word2vec**

*   ***size*** = la taille/nombre de dimensions des vecteurs de plongements générés par le modele. (idealement entre quelques dizaines a quelques centaines). Pour notre modele, on a choisit une valeur multiple de 2 pour une meuilleure gestion de memoire. De plus, les vecteurs de plongements finaux seront de taille (N, 256) avec N la taille du vocabulaire. Ainsi, on a jugé que 256 est un bon compromis.

*   ***min_count*** = la fréquence minimale des mots a considerer. Le modele ignore tous les mots du corpus dont la fréquence est inférieure a *min_count*. On a fixé cette valeur a 1 pour pouvoir construire le vocabulaire le plus large possible qui contient tous les types du corpus.

*   ***window*** = la taille de la fenetre a considerer autour du mot en question (entre le mot cible et ces voisins). (en generale entre 2 et 10). on a utiliser la valeur 5 comme juste milieu de l'intervalle recommendé.

*   ***sample*** =  le seuil de sous-echantillonnage aléatoire des mots les plus fréquents. (idealement entre 0, 1e-5). 

*   ***alpha*** = le taux d'apprentissage. Ce parametre doit etre assez petit pour pouvoir s'approcher le plus de l'optimum local, mais assez grand pour eviter le surapprentissage. Pour cela, on l'a fixé a 0.01.

*   ***min_alpha*** = la valeur a laquelle le taux d'apprentissage *alpha* va diminuer lineairement lors de l'entrainement. Une bonne estimation: alpha - (min_alpha * epochs) ~ 0.00. Dans notre cas, nos valeurs choisies respectent bien cette equation: 0.01 - 0.0001 * 10 = 0.009.

*   ***negative*** = si positive, la valeur indique le nombre de mots "bruit" a introduire. (generalement entre 5 et 20). Ce parametre permet, entre autre, d'eviter le surapprentissage. Apres plusieurs tests, on a gardé la valeur 5.

*   ***workers*** = nombre de threads a utiliser pour l'entrainement. Puisqu'on a utilisé Google Colab, les ressources alloués par session varient, donc pour s'assurer qu'on utilise le maximum de threads disponibles, on recupere ce nombre a travers *os.cpu_count()*.

## c) Décrire le cas échéant et de manière précise tout problème que vous avez eu à obtenir votre modèle et les façons de résoudre ces problèmes.

*   le parametre *sample*: Ce parametre est par excellence le plus sensible. On a du experimenté avec plusieurs valeurs pour etudier son impact et aboutir a de bons resultats.

## d) Retrouvez les 5 mots voisins des mots suivants : excellent, terrible

In [16]:
print(word_vectors.most_similar('excellent')[:5])
print(word_vectors.most_similar('terrible')[:5])

[('outstanding', 0.9267053604125977), ('fine', 0.9202706217765808), ('superb', 0.9140844345092773), ('terrific', 0.9097084999084473), ('fantastic', 0.9078267216682434)]
[('horrible', 0.9704045057296753), ('awful', 0.9541709423065186), ('suck', 0.8495052456855774), ('atrocious', 0.8381000757217407), ('baaaaaad', 0.8236089944839478)]


In [17]:
# a couple more test cases

print(word_vectors.most_similar('fun')[:5])
print(word_vectors.most_similar('film')[:5])
print(word_vectors.most_similar('acting')[:5])

[('laugh', 0.8005730509757996), ('enjoy', 0.7648371458053589), ('enjoyable', 0.752558708190918), ('entertaining', 0.7490490674972534), ('scary', 0.7229304313659668)]
[('movie', 0.7168675065040588), ('cinema', 0.7107754349708557), ('documentary', 0.7094526290893555), ('picture', 0.6874585151672363), ('flick', 0.6587791442871094)]
[('writing', 0.7815835475921631), ('directing', 0.7600515484809875), ('casting', 0.7400418519973755), ('direction', 0.7393010854721069), ('script', 0.7318971157073975)]




---



---



In [ ]:
del model # to free up space

# 2. Classification avec des plongements lexicaux

On vous demande d’effectuer de la classification avec les plongements lexicaux obtenus.

## a) En reprenant le code développé dans le TP1 avec Scikitlearn, on vous demande cette fois de tester un modèle Naïve Bayes et de régression logistique avec des n-grammes (n=1,2,3 ensemble). Essayez de voir si une réduction de dimension améliore la classification. Ne fournissez que votre meilleur modèle. Evaluez vos algorithmes selon les métriques d’accuracy générale et de F1 par classe sur l’ensemble de test.

In [13]:
def build_voc(X_train, vocab_size):
    freqs = Counter(word for doc in X_train for word in doc.split(" "))
    vocab = {word: freq for word, freq in freqs.most_common(vocab_size)}
    return {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1], reverse=True)}

print(build_voc(X_train, 500))

{'br': 101871, 'movie': 51715, 'film': 47048, 'one': 27748, 'like': 20747, 'time': 15966, 'good': 15204, 'character': 14183, 'story': 13171, 'even': 12655, 'get': 12515, 'would': 12436, 'make': 12229, 'see': 12016, 'really': 11738, 'well': 10788, 'scene': 10596, 'much': 9764, 'people': 9387, 'bad': 9309, 'also': 9159, 'great': 9090, 'first': 9065, 'way': 8830, 'show': 8603, 'made': 8364, 'thing': 8213, 'life': 8038, 'could': 7922, 'think': 7734, 'go': 7602, 'watch': 7096, 'know': 7068, 'two': 6908, 'actor': 6877, 'year': 6874, 'plot': 6874, 'love': 6856, 'seen': 6681, 'many': 6675, 'end': 6635, 'look': 6561, 'say': 6506, 'acting': 6495, 'never': 6485, 'little': 6438, 'best': 6420, 'man': 6016, 'ever': 5992, 'better': 5742, 'take': 5702, 'come': 5673, 'still': 5656, 'work': 5653, 'part': 5222, 'director': 5124, 'find': 5080, 'something': 5076, 'back': 5045, 'want': 4990, 'give': 4953, 'lot': 4779, 'real': 4737, 'performance': 4720, 'woman': 4625, 'watching': 4608, 'old': 4580, 'though':

**Sans reduction de dimensions**

In [11]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print('X_train_tfidf:', X_train_tfidf.shape)
print('X_test_tfidf:', X_test_tfidf.shape)

X_train_tfidf: (25000, 4472874)
X_test_tfidf: (25000, 4472874)


**Naive Bayes (sans reduction de dimensions)**


In [12]:
model = MultinomialNB(alpha=0.6)

model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)

print("Classification report:\n")
print(classification_report(y_test, y_pred, target_names=["N", "P"]))
print("\nAccuracy generale: %f \n" % accuracy_score(y_test, y_pred))

# plot_confusion_matrix(model, X_test_tfidf, y_test, display_labels=["N", "P"], cmap=plt.cm.Blues)
# plt.show()

Classification report:

              precision    recall  f1-score   support

           N       0.85      0.89      0.87     12500
           P       0.88      0.84      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000


Accuracy generale: 0.863480 



**Regression logistique (sans reduction de dimensions)**



In [13]:
model = LogisticRegression(C=2.0)

model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)

print("Classification report:\n")
print(classification_report(y_test, y_pred, target_names=["N", "P"]))
print("\nAccuracy generale: %f \n" % accuracy_score(y_test, y_pred))

# plot_confusion_matrix(model, X_test_tfidf, y_test, display_labels=["N", "P"], cmap=plt.cm.Blues)
# plt.show()

Classification report:

              precision    recall  f1-score   support

           N       0.88      0.87      0.88     12500
           P       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000


Accuracy generale: 0.878480 



**Avec reduction de dimensions**


In [12]:
def lsa(df, k):
    # print(df)
    U, s, V = np.linalg.svd(df, full_matrices=False)
    S = np.diag(s[:k])
    Ck = np.dot(U[:, :k], S)
    df_r =  pd.DataFrame(Ck).T
    df_r.columns = df.keys()
    return  df_r

In [16]:
X_train_lsa = lsa(np.ndarray(X_train_tfidf), 100)
# X_test_lsa = lsa(X_test_tfidf, 100)

TypeError: ignored

In [ ]:
svd = TruncatedSVD(n_components=300)
# df_r =  pd.DataFrame(df_r).T
# df_r.columns = df.keys()

X_train_lsa = svd.fit_transform(X_train_tfidf)
# X_test_lsa = svd.fit(X_test_tfidf)

**Naive Bayes (avec reduction de dimensions)**


In [ ]:
model = MultinomialNB(alpha=0.6)

model.fit(X_train_lsa, y_train)
y_pred = model.predict(X_test_lsa)

print("Classification report:\n")
print(classification_report(y_test, y_pred, target_names=["N", "P"]))
print("\nAccuracy generale: %f \n" % accuracy_score(y_test, y_pred))

# plot_confusion_matrix(model, X_test_tfidf, y_test, display_labels=["N", "P"], cmap=plt.cm.Blues)
# plt.show()

**Regression logistique (avec reduction de dimensions)**

In [ ]:
model = LogisticRegression(C=2.0)

model.fit(X_train_lsa, y_train)
y_pred = model.predict(X_test_lsa)

print("Classification report:\n")
print(classification_report(y_test, y_pred, target_names=["N", "P"]))
print("\nAccuracy generale: %f \n" % accuracy_score(y_test, y_pred))

# plot_confusion_matrix(model, X_test_tfidf, y_test, display_labels=["N", "P"], cmap=plt.cm.Blues)
# plt.show()

## b) En utilisant Tensorflow (ou Pytorch), on vous demande de développer un classificateur perceptron multicouches et un bi-LSTM avec les vecteurs d’un modèle word2vec pré-entrainé sur Wikipédia en Anglais (enwiki_upos_skipgram_300_3_2019) disponible à http://vectors.nlpl.eu/repository/11/3.zip. 

On s’attend à ce que vous effectuiez une moyenne des vecteurs de mots de chaque document pour obtenir un plongement du document.  

Evaluez vos algorithmes selon les métriques d’accuracy générale et de F1 par classe sur l’ensemble de test. Pour chacun des modèles, indiquez ses performances et ses spécifications (nombre d’époques, régularisation, optimiseur, nombre de couches, etc.). N’hésitez pas à expérimenter avec différents paramètres. Vous ne devez reporter que votre meilleure expérimentation.

## c) Ré-entrainez les modèles en b) avec vos propres vecteurs. Comparez maintenant la performance obtenue en en b) avec celles que vous obtenez en utilisant vos propres vecteurs de mots entrainés sur le corpus. 

## d) Générez une table ou un graphique qui regroupe les performances des modèles, leurs spécifications, la durée d’entraînement et commentez ces résultats. Quelle est l’influence des word embeddings sur les performances?  Quel est votre meilleur modèle ?